<a href="https://colab.research.google.com/github/fallensnake/Kalshi-Bots/blob/main/FirstKalshibot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas statsmodels kalshi-python-sync matplotlib

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

# 1. Load Data (CRITICAL: Use Seasonally Adjusted Data)
# Download 'CPIAUCSL.csv' from FRED: https://fred.stlouisfed.org/series/CPIAUCSL
def load_and_prep_data(filepath):
    df = pd.read_csv(filepath)
    df['Date'] = pd.to_datetime(df['Date']) # FRED usually uses 'observation_date'
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    df = df.asfreq('MS')
    return df

# 2. Train Model
def train_forecast_model(df):
    # Log transform to stabilize variance (standard for CPI)
    df['log_cpi'] = np.log(df['CPI'])

    # ARIMA(1,1,1) is a decent simple baseline for log-CPI
    # We use the log data, so d=1 becomes percentage change
    model = ARIMA(df['log_cpi'], order=(1, 1, 1))
    model_fit = model.fit()

    # Forecast 1 step ahead (for Dec 2025)
    forecast_result = model_fit.get_forecast(steps=1)
    pred_log_cpi = forecast_result.predicted_mean.iloc[0]

    # Convert back from log to actual price level
    pred_cpi_level = np.exp(pred_log_cpi)

    return pred_cpi_level

# 3. Calculate MoM with BLS Rounding Rules
def calculate_official_mom(last_actual_level, predicted_level):
    # Calculate raw percent change
    raw_change = ((predicted_level - last_actual_level) / last_actual_level) * 100

    # BLS Rounding Logic: Standard rounding to 1 decimal place
    # Note: Python's round() finds nearest even number for .5, so we use a custom approach
    # or just standard formatted string for simplicity which mimics standard rounding often enough for baselines.
    rounded_change = float(f"{raw_change:.1f}")

    return raw_change, rounded_change

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
if __name__ == "__main__":
    # Ensure your CSV is named correctly and is the SEASONALLY ADJUSTED version
    data = load_and_prep_data('/content/drive/MyDrive/KalshiBot1/US_CPI.csv')

    # Get last known value (November 2025 data)
    last_actual_cpi = data['CPI'].iloc[-1]
    last_date = data.index[-1]

    print(f"Latest Data Point: {last_date.strftime('%b %Y')} = {last_actual_cpi}")

    if last_date.month != 11:
        print("⚠️ WARNING: Your dataset does not seem to have November data yet.")

    # Predict December
    pred_cpi = train_forecast_model(data)

    # Calculate the number the market trades on
    raw_mom, official_mom = calculate_official_mom(last_actual_cpi, pred_cpi)

    print("-" * 40)
    print(f"Forecast for Dec 2025 (Released Jan 2026)")
    print(f"Predicted Index Level: {pred_cpi:.3f}")
    print(f"Raw MoM Change:      {raw_mom:.4f}%")
    print(f"OFFICIAL MoM (Rounded): {official_mom}%")
    print("-" * 40)

    thresholds = [-0.1, 0.0, 0.1, 0.2, 0.3]
    print("\nMarket Prediction Check:")
    for t in thresholds:
        status = "YES" if official_mom > t else "NO"
        print(f"Above {t}%? -> {status}")

Latest Data Point: Nov 2025 = 325.031
----------------------------------------
Forecast for Dec 2025 (Released Jan 2026)
Predicted Index Level: 325.340
Raw MoM Change:      0.0951%
OFFICIAL MoM (Rounded): 0.1%
----------------------------------------

Market Prediction Check:
Above -0.1%? -> YES
Above 0.0%? -> YES
Above 0.1%? -> NO
Above 0.2%? -> NO
Above 0.3%? -> NO


In [5]:
import uuid